In [2]:
import numpy as np


from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

from keras.optimizers import Adam

In [1]:
import tensorflow as tf
from keras import backend as K

num_cores = 6


num_CPU = 1
num_GPU = 0

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [11]:
X = np.random.random((1000, 4, 42, 42))
Y = np.random.random((1000, 6))

In [4]:
shared_layers = [
    Conv2D(16, kernel_size=8, strides=4, activation='relu', data_format='channels_first', input_shape=(4, 42, 42)),
    Conv2D(32, kernel_size=4, strides=2, activation='relu'),
    Flatten(),
    Dense(256, activation='relu')
]
models = [Sequential(shared_layers + [Dense(1)]) for _ in range(6)]

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
opt = Adam(lr=0.1,
           # beta_1=0.9, beta_2=0.999, epsilon=None,
           #decay=0.01,
           amsgrad=False)
for model in models:
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_absolute_error'])

In [7]:
from sklearn.metrics import mean_squared_error as mse
mse(Y, models[0].predict(X))

0.21209712386468366

In [12]:
for i in range(6):
    print(f'i: {i}, mse: {mse(Y[:, i], models[i].predict(X))}')

i: 0, mse: 0.1892761624708104
i: 1, mse: 0.3059353547420307
i: 2, mse: 0.3801952001259136
i: 3, mse: 0.484629219980543
i: 4, mse: 0.6038949231902593
i: 5, mse: 0.38593367767336645


In [20]:
for i in range(6):
    print(f'i: {i}, mse: {mse(Y[:, i], models[i].predict(X))}')

i: 0, mse: 0.38818935641557756
i: 1, mse: 0.33095162526326927
i: 2, mse: 0.33608280549823794
i: 3, mse: 0.31954324784754773
i: 4, mse: 0.33306724521545583
i: 5, mse: 0.33957934150394053


In [18]:
models[0].train_on_batch(X, Y[:, 0])

[0.33790183, 0.5057303]